In [1]:
#LOADACTION --- loads the MoCap streams of an action specified by the user.
# Input: 
#       - folder: path of the folder in which mat structures 
#                       are stored.
#       - mode: 'training' or 'test'
#       - action: name of the action that needs to be loaded.
#       - output_folder: folder name to save data and index files
#       - marker (optional): a single string between 'SHOULDER',
#                       'ELBOW', 'WRIST', 'PALM', 'IND' and 'LIT'.
#                       'ALL' to retrieve MoCap data from each marker.
#       - instance (optional): a number identifying a single
#                       instance of the action.
#              
#       This function will give you the possibility to load only
#       part of the Cooking Dataset by specifying and action, a
#       marker, or an instance.
#              
# Output:
#         data file  

# action=['carrot','cutting','dish','eating','eggs','lemon','mezzaluna','mixing','openbottle','pan','pestling','pouringmult','pouringsingle','reaching','rolling','salad','salting','spreading','table','transporting']
# markers
#    -IND(Index Finger)
#    -PALM
#    -LIT(Little Finger)
#    -WRIST
#    -ELBOW
#    -SHOULDER


# Example of use:
# folder = '../CookingDataset-master/data/mocap/';
# mode="training" # or 'test'
# action="carrot"
# output_folder="./data/loadAction/"
#
#  - loadAction(folder,mode, action,output_folder) and 
#    loadAction(folder,mode, action,output_folder, 'ALL') 
#   file named *action*_ALLmarker_ALLinstance.csv  will be saved
# 	%save data containing all the markers for the action specified.
# 
#
#  - loadAction(folder,mode, action,output_folder, marker)
#   file named *action*_*marker*_ALLinstance.csv  will be saved
# 	%save data containing specific marker for the action.
#
#  - loadAction(folder,mode, action,output_folder, 'ALL', instance) 
#   file named *action*_*marker*_ALLmarker_*instance*.csv  will be saved
#   %save data containing all markers at the specified instance of  the action.
#
#  - loadAction(folder,mode, action,output_folder, marker, instance) 
#   file named *action*_*marker*_*marker*_*instance*.csv  will be saved
#    % save data of a single marker at the specifies instance of the action.

import os
from os import path
import sys
import csv
import pandas as pd
from scipy.io import loadmat 

def loadAction(folder, mode, action,output_folder, *args):
    all_column_names=['IND_X','IND_Y','IND_Z','PALM_X','PALM_Y','PALM_Z', 'LIT_X', 'LIT_Y','LIT_Z','WRIST_X','WRIST_Y','WRIST_Z', 'ELBOW_X','ELBOW_Y','ELBOW_Z', 'SHOULDER_X','SHOULDER_Y','SHOULDER_Z']
    all_marker = ['IND','PALM', 'LIT', 'WRIST', 'ELBOW', 'SHOULDER']
    if not os.path.exists(output_folder):#create output path if not exist
        os.makedirs(output_folder)
    
    if mode == 'training':
        file = path.join(folder,mode,action)+'_tr.mat'
        
    elif mode == 'test':
        file = path.join(folder,mode,action)+'_test.mat'
        

    print("Matlab file name:",file)
    single_action = loadmat(file)  # load mat-file
    length=len(args)
    print("Optional Arguments:",length)

    
    if length == 0 :
        f=path.join(output_folder,(action+'_ALLmarker_ALLinstance.csv'))
        column_names =all_column_names
        marker=all_marker
        all_action(file,f,column_names,marker)



        
    elif length == 1:
        marker= args[0]
        
        if marker == 'ALL':
            f=path.join(output_folder,(action+'_ALLmarker_ALLinstance.csv'))
            column_names =all_column_names
            marker=all_marker
            all_action(file,f,column_names,marker)
            
        else:
            column_names=[marker+'_X',marker+'_Y',marker+'_Z']
            new_marker=[marker]
            f=path.join(output_folder,(action+'_'+marker+'_ALLinstance.csv'))
            all_action(file,f,column_names,new_marker)

        
    elif length ==2:
        marker= args[0]
        instance =args[1]

        if marker == 'ALL':
            marker=all_marker
            column_names =all_column_names
            f=path.join(output_folder,(action+'_'+'ALLmarker'+'_'+str(instance)+'.csv'))
            all_action_index(file, f,column_names,marker,instance)

        else:
            column_names=[marker+'_X',marker+'_Y',marker+'_Z']
            new_marker=[marker]
            f=path.join(output_folder,(action+'_'+marker+'_'+str(instance)+'.csv'))
            all_action_index(file,f,column_names,new_marker,instance)
    

def all_action(file, f,column_names,marker):
    df = pd.DataFrame(columns = column_names)
    single_action = loadmat(file) 
        
    for a in marker:
            arr = single_action[a]
            x=a+'_X'
            y=a+'_Y'
            z=a+'_Z'
            df[x]=arr[:,0]
            df[y]=arr[:,1]
            df[z]=arr[:,2]
    df.to_csv(f,index = False)
    
def all_action_index(file, f,column_names,marker,instance):
    df = pd.DataFrame(columns = column_names)
    single_action = loadmat(file) 
    index=single_action['index']
    index_length=len(index[0])
    print("Index Length:",index_length)
    print("Number of Instance:",index_length-1)
    start= index[0][instance-1]
    end=index[0][instance]
    print("Start index of Instance:",start)
    print("End index of Instance:",end)
    

    for a in marker:
            arr = single_action[a]

            x=a+'_X'
            y=a+'_Y'
            z=a+'_Z'
            df[x]=arr[:,0]
            df[y]=arr[:,1]
            df[z]=arr[:,2]
    
    df=df.iloc[start-1:end]
    df.to_csv(f,index = False)

In [5]:
# Function Calling
folder = "../CookingDataset-master/data/mocap/"  # cooking dataset folder
mode="training" # or 'test'
action="carrot"
output_folder="./data/loadAction/"
marker="ELBOW"
instance=7

loadAction(folder,mode,action,output_folder)
# loadAction(folder,mode,action,output_folder,marker)
# loadAction(folder,mode,action,output_folder,marker,instance)

Matlab file name: ../CookingDataset-master/data/mocap/training/carrot_tr.mat
Optional Arguments: 0


# Visualize Results

In [3]:
#output_folder files
print(os.listdir(output_folder))

['carrot_ALLmarker_7.csv', 'carrot_ELBOW_7.csv']


In [4]:
#DATA
data_filename=path.join(output_folder,'carrot_ELBOW_7.csv')
df=pd.read_csv(data_filename)
print("shape:",df.shape)
df

shape: (44, 3)


,ELBOW_X,ELBOW_Y,ELBOW_Z
0,-67.679150,283.61130,85.215065
1,-67.985830,284.09110,84.041120
2,-68.126960,284.26642,83.687000
3,-67.916950,284.19016,83.880020
4,-67.876724,284.02176,84.633130
5,-68.155365,283.91970,85.787346
6,-68.527110,283.89886,87.201330
7,-69.061070,283.95712,88.704240
8,-69.743050,284.08620,90.254970
9,-70.323296,284.12640,91.900055
